In [2]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
# Import the required message types
from langchain_core.messages import AIMessage, ToolMessage

# Load API keys and set up tracing
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Intro to LangGraph"

# --- Define Tools ---
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

# --- Define State and Nodes ---
class GraphState(TypedDict):
    messages: list

def call_llm(state: GraphState):
    llm = ChatOpenAI(model="gpt-4o")
    messages = state['messages']
    llm_with_tools = llm.bind_tools([multiply])
    response = llm_with_tools.invoke(messages)
    return {"messages": state['messages'] + [response]}

def call_tool(state: GraphState):
    last_message = state['messages'][-1]
    tool_call = last_message.tool_calls[0]
    tool_name = tool_call['name']
    tool_args = tool_call['args']
    
    if tool_name == "multiply":
        result = multiply.invoke(tool_args)

    # FIX: Wrap the integer result in a ToolMessage
    # The content must be a string, and you must include the tool_call_id
    tool_message = ToolMessage(
        content=str(result), 
        tool_call_id=tool_call['id']
    )
    
    return {"messages": state['messages'] + [tool_message]}

# --- Define the Router (Conditional Edge) ---
def should_call_tool(state: GraphState) -> Literal["call_tool", "__end__"]:
    last_message = state['messages'][-1]
    if last_message.tool_calls:
        return "call_tool"
    return "__end__"

# --- Build the Graph ---
workflow = StateGraph(GraphState)
workflow.add_node("llm", call_llm)
workflow.add_node("call_tool", call_tool)
workflow.set_entry_point("llm")
workflow.add_conditional_edges(
    "llm",
    should_call_tool,
)
workflow.add_edge("call_tool", "llm")
app = workflow.compile()

# Run the app. The input must be a list of Message objects.
final_state = app.invoke({"messages": [("user", "What is 4 * 5?")]})

# Print the final AI message
print(final_state['messages'][-1].content)

4 multiplied by 5 is 20.
